In [1]:

import pandas as pd
import numpy as np
import math

data = pd.read_csv('3-data.csv')
print(data)


    ID  年龄 有工作 有自己的房子 信贷情况 类别
0    1  青年   否      否   一般  否
1    2  青年   否      否    好  否
2    3  青年   是      否    好  是
3    4  青年   是      是   一般  是
4    5  青年   否      否   一般  否
5    6  中年   否      否   一般  否
6    7  中年   否      否    好  否
7    8  中年   是      是    好  是
8    9  中年   否      是  非常好  是
9   10  中年   否      是  非常好  是
10  11  老年   否      是  非常好  是
11  12  老年   否      是    好  是
12  13  老年   是      否    好  是
13  14  老年   是      否  非常好  是
14  15  老年   否      否   一般  否


In [2]:

def get_emp_entropy(arr):
    '''
    经验熵
    :param arr:
    :return:
    '''
    dict = {}
    for a in arr:
        if a in dict:
            dict[a] += 1
        else:
            dict[a] = 1
    # print(dict)
    result = 0
    for k, v in dict.items():
        p = v / len(arr)
        result += p * math.log(p, 2)
    result = -1.0 * result
    return result

In [3]:

def get_emp_cond_entropy(arr_feature, classes):
    '''
    经验条件熵
    :return:
    '''
    dict = {}
    index = 0
    classes = list(classes)
    for a in arr_feature:
        if a in dict:
            dict[a].append(classes[index])
        else:
            dict[a] = list(classes[index])
        index += 1
    # print(dict)

    result = 0
    for k, v in dict.items():
        p = len(v) / len(arr_feature)
        result += p * get_emp_entropy(v)
    return result

In [4]:

def gain(arr_feature, classes):
    '''
    信息增益
    '''
    g = get_emp_entropy(classes) - get_emp_cond_entropy(arr_feature, classes)
    return g

In [5]:

classes = data['类别']
H_D = get_emp_entropy(classes)
print("【类别】的经验熵：{}".format(H_D))

【类别】的经验熵：0.9709505944546686


In [6]:

g_Age = gain(data['年龄'], classes)
print("【年龄】的信息增益：{}".format(g_Age))

【年龄】的信息增益：0.08300749985576883


In [7]:

g_Work = gain(data['有工作'], classes)
print("【有工作】的信息增益：{}".format(g_Work))

【有工作】的信息增益：0.32365019815155627


In [8]:

g_House = gain(data['有自己的房子'], classes)
print("【有自己的房子】的信息增益：{}".format(g_House))

【有自己的房子】的信息增益：0.4199730940219749


In [9]:

g_Credit = gain(data['信贷情况'], classes)
print("【信贷情况】的信息增益：{}".format(g_Credit))


【信贷情况】的信息增益：0.36298956253708536


In [10]:
gains=[]
features=[]

gains.append(g_Age)
features.append("年龄")

gains.append(g_Work)
features.append("有工作")

gains.append(g_House)
features.append("有自己的房子")

gains.append(g_Credit)
features.append("信贷情况")

best=np.max(gains)
index = gains.index(best)
print('第一次选择, 最优特征：{}'.format(features[index]))

第一次选择, 最优特征：有自己的房子


In [11]:
# 划分数据集
D1 = data[data['有自己的房子'].isin( ['是'])]
D2 = data[data['有自己的房子'].isin( ['否'])]
print('有自己的房子：是')
print(D1)
print('有自己的房子：否')
print(D2)

有自己的房子：是
    ID  年龄 有工作 有自己的房子 信贷情况 类别
3    4  青年   是      是   一般  是
7    8  中年   是      是    好  是
8    9  中年   否      是  非常好  是
9   10  中年   否      是  非常好  是
10  11  老年   否      是  非常好  是
11  12  老年   否      是    好  是
有自己的房子：否
    ID  年龄 有工作 有自己的房子 信贷情况 类别
0    1  青年   否      否   一般  否
1    2  青年   否      否    好  否
2    3  青年   是      否    好  是
4    5  青年   否      否   一般  否
5    6  中年   否      否   一般  否
6    7  中年   否      否    好  否
12  13  老年   是      否    好  是
13  14  老年   是      否  非常好  是
14  15  老年   否      否   一般  否


In [12]:
# 第二轮选择信息增益

classes2 = D2['类别']
H_D2 = get_emp_entropy(classes2)

gains = []
features = []

G_D2_Age = H_D2 - get_emp_cond_entropy(D2['年龄'], classes2)
print("【{}】的信息增益：{}".format('年龄', G_D2_Age))
gains.append(G_D2_Age)
features.append('年龄')

G_D2_Work = H_D2 - get_emp_cond_entropy(D2['有工作'], classes2)
print("【{}】的信息增益：{}".format('有工作', G_D2_Work))
gains.append(G_D2_Work)
features.append('有工作')

G_D2_House = H_D2 - get_emp_cond_entropy(D2['信贷情况'], classes2)
print("【{}】的信息增益：{}".format('信贷情况', G_D2_House))
gains.append(G_D2_House)
features.append('信贷情况')

best = np.max(gains)
index = gains.index(best)
print('第二次选择，最优特征：{}'.format(features[index]))

【年龄】的信息增益：0.2516291673878229
【有工作】的信息增益：0.9182958340544896
【信贷情况】的信息增益：0.47385138961004514
第二次选择，最优特征：有工作


In [13]:
def gain_ratio(arr_feature, classes):
    '''
    信息增益比
    :param arr_feature:
    :param classes:
    :return:
    '''
    g_ratio = gain(arr_feature, classes) / get_emp_entropy(classes)
    return g_ratio